In [ ]:
import os
from dotenv import load_dotenv

from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain_google_genai import GoogleGenerativeAIEmbeddings, ChatGoogleGenerativeAI
from langchain.chains import RetrievalQA

In [ ]:
load_dotenv()
google_api_key = os.getenv("GOOGLE_API_KEY")

In [ ]:
pdf_folder = "RAGData"
documents = []

In [ ]:
for file in os.listdir(pdf_folder):
    if file.endswith(".pdf"):
        loader = PyPDFLoader(os.path.join(pdf_folder, file))
        docs = loader.load()
        documents.extend(docs)
        print(f"Loaded {len(docs)} pages from {file}")

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50
)

In [ ]:
split_docs = text_splitter.split_documents(documents)

In [ ]:
embedding = GoogleGenerativeAIEmbeddings(
    model="models/embedding-001",
    google_api_key=google_api_key
)

In [ ]:
vectorstore_dir = "./chroma_db"

In [ ]:
if os.path.exists(os.path.join(vectorstore_dir, "index")):
    print("Loading existing Chroma vectorstore...")
    vectorstore = Chroma(persist_directory=vectorstore_dir, embedding=embedding)
else:
    print("Creating new Chroma vectorstore...")
    vectorstore = Chroma.from_documents(
        documents=split_docs,
        embedding=embedding,
        persist_directory=vectorstore_dir
    )

In [ ]:
llm = ChatGoogleGenerativeAI(
    model="models/gemini-1.5-pro-latest",  # You may switch to gemini-pro if hitting limits
    google_api_key=google_api_key
)

In [ ]:
retriever = vectorstore.as_retriever()

In [ ]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True
)

In [ ]:
query = "What does 'payment account' mean?"

In [15]:
try:
    response = qa_chain.invoke(query)
    print("\n📌 Answer:\n", response['result'])

    print("\n📄 Source Documents:")
    for i, doc in enumerate(response['source_documents']):
        print(f"\nSource {i+1} - {doc.metadata.get('source', 'Unknown')}:\n{doc.page_content[:300]}...")

except Exception as e:
    print("❌ An error occurred:", e)

Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerDayPerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-1.5-pro"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
}
violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerMinutePerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-1.5-pro"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
}
violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_

❌ An error occurred: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_input_token_count"
  quota_id: "GenerateContentInputTokensPerModelPerMinute-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-1.5-pro"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
}
violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerMinutePerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-1.5-pro"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
}
violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerDayPerProjectPerModel-FreeTier"
  quota_dimension